In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy.stats import pearsonr
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

from numba import jit, float64, int64

In [2]:
@jit(nopython=True)
def simulate_CDDM(threshold, lamda, delta, loc, scale, ndt=0, z=0, sigma=1, dt=0.001):
    x = z
    
    rt = 0
    
    while -(threshold / (1 + lamda*rt)) < x and x < (threshold / (1 + lamda*rt)):
        x += delta * dt + sigma*np.sqrt(dt)*np.random.normal(0, 1)
        rt += dt
        
    if x >= (threshold / (1 + lamda*rt)):
        ch = 1
    else:
        ch = -1
        
    return (rt+ndt)*ch, np.random.lognormal(loc, scale)

In [3]:
@jit(nopython=True)
def f(x, t, z, tau, delta, sigma=1):
    term1 = 1/np.sqrt(2 * np.pi * sigma**2 * (t-tau))
    term2 = -(x - z - delta * (t-tau))**2 / (2 * sigma**2 * (t-tau))
    return term1 * np.exp(term2)

@jit(nopython=True)
def psi(threshold, lamda, t, z, tau, delta, sigma=1):
    db = (-lamda*threshold)/(1 + lamda*t)**2
    term1 = 0.5*f(threshold /(1 + lamda*t), t, z, tau, delta, sigma)
    term2 = db - delta - ((threshold /(1 + lamda*t)) - z - delta * (t-tau))/(t-tau)
    return term1 * term2

@jit(nopython=True)
def fpt(threshold, lamda, delta, z=0, sigma=1, dt=0.02, T_max=5):
    gu = np.zeros((int(T_max/dt)+2,))
    gl = np.zeros((int(T_max/dt)+2,))
    T = np.zeros((int(T_max/dt)+2,))
    
    gu[1] = -2*psi(threshold, lamda, dt, z, 0, delta, sigma)
    gl[1] =  2*psi(-threshold, lamda, dt, z, 0, delta, sigma)
    T[1] = dt
    
    for n in range(2, int(T_max/dt)+2):
        su = -2 * psi( threshold, lamda, n*dt, z, 0, delta, sigma)
        sl =  2 * psi(-threshold, lamda, n*dt, z, 0, delta, sigma)
        
        for j in range(1, n):
            if (threshold /(1 + lamda*j*dt)) == 0:
                continue
            
            psi_n_j_pp = psi( threshold, lamda, n*dt,  threshold /(1 + lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_pn = psi( threshold, lamda, n*dt, -threshold /(1 + lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_np = psi(-threshold, lamda, n*dt,  threshold /(1 + lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_nn = psi(-threshold, lamda, n*dt, -threshold /(1 + lamda*j*dt), j*dt, delta, sigma)
            
            su +=  2 * dt * (gu[j] * psi_n_j_pp + gl[j] * psi_n_j_pn)
            sl += -2 * dt * (gu[j] * psi_n_j_np + gl[j] * psi_n_j_nn)
            
        gu[n] = su
        gl[n] = sl
        T[n] = (n*dt)
    return gu, gl, T

In [4]:
def CDDM_likelihood(prms, RT, Z):
    delta = prms[2]
    sig = prms[3]
    t0 = prms[4]
    
    T_max = np.max(np.abs(RT))
    gu, gl, TT = fpt(prms[0], prms[1], delta, z=0, dt=0.02, T_max=T_max)
    
    gtup = interp1d(TT, gu)
    gtlp = interp1d(TT, gl)
    
    ll = 0
    for i in range(len(RT)):
        if np.abs(RT[i])-t0 > 0:
            if RT[i]>=0:
                
                ll += 0.5*(np.log(Z[i]) - np.log(t0) + 0.5*sig**2)**2/sig**2 + 0.5*np.log(2*np.pi*sig**2*Z[i]**2)
                
                if gtup(RT[i])>1e-14:
                    ll += -np.log(gtup(np.abs(RT[i])-t0))
                else:
                    ll += -np.log(1e-14)
            else:
                
                ll += 0.5*(np.log(Z[i]) - np.log(t0) + 0.5*sig**2)**2/sig**2 + 0.5*np.log(2*np.pi*sig**2*Z[i]**2)
                
                if gtlp(np.abs(RT[i])-t0)>1e-14:
                    ll += -np.log(gtlp(np.abs(RT[i])-t0))
                else:
                    ll += -np.log(1e-14) 
        else:
            ll += -np.log(1e-14)
    
    return ll

def DDM_likelihood(prms, RT, Z):
    delta = prms[1]
    sig = prms[2]
    t0 = prms[3]
    
    T_max = np.max(np.abs(RT))
    gu, gl, TT = fpt(prms[0], 0, delta, z=0, dt=0.02, T_max=T_max)
    
    gtup = interp1d(TT, gu)
    gtlp = interp1d(TT, gl)
    
    ll = 0
    for i in range(len(RT)):
        if np.abs(RT[i])-t0 > 0:
            if RT[i]>=0:
                
                ll += 0.5*(np.log(Z[i]) - np.log(t0) + 0.5*sig**2)**2/sig**2 + 0.5*np.log(2*np.pi*sig**2*Z[i]**2)
                
                if gtup(RT[i])>1e-14:
                    ll += -np.log(gtup(np.abs(RT[i])-t0))
                else:
                    ll += -np.log(1e-14)
            else:
                
                ll += 0.5*(np.log(Z[i]) - np.log(t0) + 0.5*sig**2)**2/sig**2 + 0.5*np.log(2*np.pi*sig**2*Z[i]**2)
                
                if gtlp(np.abs(RT[i])-t0)>1e-14:
                    ll += -np.log(gtlp(np.abs(RT[i])-t0))
                else:
                    ll += -np.log(1e-14) 
        else:
            ll += -np.log(1e-14)
    
    return ll

In [5]:
n_trials = 100
recovery_dic = {'True_model': [],
                'G2_ddm': [],
                'G2_ct_ddm': [],
                'BIC_ddm': [],
                'BIC_ct_ddm': [],
                'lambda': []}

In [6]:
for n in tqdm(range(20)):
    threshold = np.random.uniform(1.5, 4)
    if np.random.random() < 0.5:
        lamda = 0
    else:
        lamda = np.random.uniform(0.2, 2)
    
    delta = np.random.uniform(0, 1)
    ndt = np.random.uniform(0.05, 1)
    scale_z = np.random.uniform(0.1, 1)
    loc_z = np.log(ndt) - 0.5 * scale_z**2
    
    RT = []
    Z = []
    
    for i in range(n_trials):
        rt, z = simulate_CDDM(threshold, lamda, delta, loc_z, scale_z, ndt=ndt)
        RT.append(rt)
        Z.append(z)
        
    RT = np.array(RT)
    Z = np.array(Z)
    
    ans_cddm = differential_evolution(CDDM_likelihood,
                                      args=(RT, Z), 
                                      bounds=[(1.5, 4), (0, 2), (0, 3), 
                                              (0.05, 1), (0.05, 1)])
    ans_cddm = minimize(CDDM_likelihood,
                        args=(RT, Z),
                        method='Nelder-Mead',
                        x0=ans_cddm.x,
                        bounds=[(1.5, 4), (0, 2), (0, 3), 
                                (0.05, 1), (0.05, 1)])
    
    ans_ddm = differential_evolution(DDM_likelihood,
                                      args=(RT, Z), 
                                      bounds=[(1.5, 4), (0, 3), 
                                              (0.05, 1), (0.05, 1)])
    ans_ddm = minimize(DDM_likelihood,
                        args=(RT, Z),
                        method='Nelder-Mead',
                        x0=ans_ddm.x,
                        bounds=[(1.5, 4), (0, 3), 
                                (0.05, 1), (0.05, 1)])
    
    if lamda == 0:
        recovery_dic['True_model'].append('DDM')
    else:
        recovery_dic['True_model'].append('CTDDM')
    
    recovery_dic['G2_ddm'].append(2*ans_ddm.fun)
    recovery_dic['G2_ct_ddm'].append(2*ans_cddm.fun)
    recovery_dic['BIC_ddm'].append(2*ans_ddm.fun + 4*np.log(n_trials))
    recovery_dic['BIC_ct_ddm'].append(2*ans_cddm.fun + 5*np.log(n_trials))
    recovery_dic['lambda'].append(ans_cddm.x[1])

100%|███████████████████████████████████████████| 20/20 [08:25<00:00, 25.29s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_dic)
recovery_df['Best_model'] = 'CTDDM'
recovery_df.loc[recovery_df['BIC_ddm'] < recovery_df['BIC_ct_ddm'], 'Best_model'] = 'DDM'

In [8]:
file_name = '_data/Hyperbolic_ndt_info_{}.csv'.format(n_trials)
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
(recovery_df['True_model'] == recovery_df['Best_model']).sum()/recovery_df.shape[0]

0.95